In [0]:
import multiprocessing as mp
import time
from random import randint

Función wakeup

In [0]:
def wakeup(entidad):
    global producer_is_awake
    global consumer_is_awake

    if entidad == 'producer':
        producer_is_awake = True
    elif entidad == 'consumer':
        consumer_is_awake = True
    else:
        pass

Función sleep

In [0]:
def sleep(entidad):
    global producer_is_awake
    global consumer_is_awake

    if entidad == 'producer':
        producer_is_awake = False
    elif entidad == 'consumer':
        consumer_is_awake = False
    else:
        pass


# Productor

In [0]:
def produceItem():
    time.sleep(3)
    return randint(0,100)

In [0]:
def putItemIntoBuffer(item,buffer):
    print(f"{item} ha sido colocado en el buffer")
    buffer.put(item) 

In [0]:
def producer(itemCount,buffer):
    global BUFFER_SIZE
    global producer_is_awake
    global consumer_is_awake
    print("Producer")
    while True:
        if producer_is_awake:
            item = produceItem()
            print(f"Se produjo {item}")
            if itemCount.value == BUFFER_SIZE:
                print("Posible overflow, producer se duerme")
                sleep('producer')
            else:
                putItemIntoBuffer(item,buffer)
                itemCount.value += 1

        if itemCount.value == 1:
            wakeup('consumer')

# Consumidor

In [0]:
def removeItemFromBuffer(buffer):
    return buffer.get()

In [0]:
def consumeItem(item):
    time.sleep(3)
    print(f"{item} ha sido consumido")


In [0]:
def consumer(itemCount,buffer):
    global BUFFER_SIZE
    global producer_is_awake
    global consumer_is_awake
    print("Consumer")
    while True:
        if consumer_is_awake:
            if itemCount.value == 0:
                print("Posible underflow, consumer se duerme")
                sleep('consumer')
            else:
                item = removeItemFromBuffer(buffer)
                print(f"se quitó {item} del buffer")
                itemCount.value -= 1
                consumeItem(item)

        if itemCount.value < BUFFER_SIZE:
            print("Hay espacio en el buffer, producer se despierta")
            wakeup('producer')

Main

In [10]:
BUFFER_SIZE = 5
producer_is_awake = True
consumer_is_awake = True 

itemCount = mp.Value('i',5)

buffer = mp.Queue(maxsize=BUFFER_SIZE)
    
for n in [3,7,9]:
    buffer.put(n)

productor = mp.Process(target=producer, args=(itemCount,buffer))
consumidor = mp.Process(target=consumer, args=(itemCount,buffer))

productor.start()
consumidor.start()

while True:
    print(f"Hay {itemCount.value} items")
    time.sleep(3)

Producer
Consumer
se quitó 3 del buffer
Hay 5 items
Se produjo 50
50 ha sido colocado en el buffer
3 ha sido consumido
se quitó 7 del buffer
Hay 4 items
Se produjo 54
54 ha sido colocado en el buffer
7 ha sido consumido
se quitó 9 del buffer
Hay 4 items
Se produjo 37
37 ha sido colocado en el buffer
9 ha sido consumido
se quitó 50 del buffer
Hay 4 items
Se produjo 69
69 ha sido colocado en el buffer
50 ha sido consumido
se quitó 54 del buffer
Hay 4 items
Se produjo 41
41 ha sido colocado en el buffer
54 ha sido consumido
se quitó 37 del buffer
Hay 4 items


Process Process-1:
Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-4ad249ce4bd6>", line 8, in producer
    item = produceItem()
  File "<ipython-input-7-0dc8828e53ea>", line 2, in produceItem
    time.sleep(3)
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-5a0fa7445342>", line 15, in consumer
    consumeItem(item)
  File "<ipython-input-5-73467335b918>", line 2, in consumeItem
    time.sleep(3)
KeyboardInterrupt


KeyboardInterrupt: ignored